<a href="https://colab.research.google.com/github/Naznin22/Naznin22/blob/main/NLP_with_Bert_for_Sentiment_Analysis_Naznin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP with Bert for Sentiment Analysis

### Importing the libraries

In [2]:
!pip install tensorflow==2.15.1
!pip install tf_keras
!pip install ktrain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installatio

In [3]:
import os.path
# used only loading the dataset into the notebook
import numpy as np
os.environ['TF_USE_LEGACY_KERAS'] = "1"
import tensorflow as tf
# deep learning library Tensorflow
# print(tf.)


In [4]:

# ktrain is a lightweight wrapper for the deep learning library TensorFlow Keras (and other libraries) to help build, train, and deploy neural networks.
import ktrain
from ktrain import text
# text module imported from ktrain library for text preprocessing and text wrapiing into bert model

# while importing ktrain faces this error - # ImportError: `keras.optimizers.legacy` is not supported in Keras 3
# ktrain does not yet support Keras 3 (used by tensorflow>=2.16. As a temporary workaround, you can downgrade TensorFlow
# or Install a version of tf_keras compatible with your installed tensorflow version: pip install tf_keras # installs latest
# Set the TF_USE_LEGACY_KERAS environment variable to True: os.environ['TF_USE_LEGACY_KERAS'] = "1"

## Part 1: Data Preprocessing
Bert model has no knowledge in English language. SO we'll train it with the positive and negative text reviews


### Loading the IMDB dataset



In [5]:
# load dataset with keras library
dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz",
                                  origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
                                  extract=True)
DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')

84125825/84125825 [==============================] - 17s 0us/step


In [6]:
# path leadin to the dataset of the keras
print(os.path.dirname(dataset))

#
print(DATADIR)

/root/.keras/datasets
/root/.keras/datasets/aclImdb


### Creating the training and test sets

In [7]:
(x_train, y_train), (x_test, y_test), prepoc = text.texts_from_folder(datadir=DATADIR,
                       classes = ['pos','neg'],
                       maxlen = 500,
                       train_test_names = ['train','test'],
                       preprocess_mode = 'bert')
# set maxlen for time saving of training

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## Part 2: Building the BERT model

In [8]:
 bert_model = text.text_classifier(name = 'bert',
                                  train_data = (x_train, y_train),
                                  preproc = prepoc)



Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


done.


In [9]:
print(bert_model)
if bert_model is None:
  print("Error: bert_model is None. Check text.text_classifier function.")
else:
  print("ok")

ok


## Part 3: Training the BERT model

In [10]:
#part 1 -get the learner part of the model
#part 2 - launch the training on one cycle using fitonecycle function from the k train library
learner_instance = ktrain.get_learner(bert_model,
                                      train_data = (x_train, y_train),
                                      val_data = (x_test, y_test),
                                      batch_size = 6
                                      )


In [11]:
# learning rate recommended for one epoch in one cycle for maxlen 500 of batch size 6 is 2*10^-5
learner_instance.fit_onecycle(lr = .00002,
                              epochs = 1,
                              )



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 4604s 1s/step - loss: 0.2474 - accuracy: 0.8983 - val_loss: 0.1643 - val_accuracy: 0.9384




begin training using onecycle policy with max lr of 2e-05...
  28/4167 [..............................] - ETA: 52:54 - loss: 0.7158 - accuracy: 0.5000

  

begin training using onecycle policy with max lr of 2e-05...
 144/4167 [>.............................] - ETA: 52:34 - loss: 0.6362 - accuracy: 0.6296



begin training using onecycle policy with max lr of 2e-05...
 609/4167 [===>..........................] - ETA: 48:00 - loss: 0.4023 - accuracy: 0.8090


begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - ETA: 0s - loss: 0.2474 - accuracy: 0.8983